#### Paracode

Laster ned moduler og filer, `module_update`, `nbtext` og `graph_networkx_louvain`

In [ ]:
! curl -s "https://raw.githubusercontent.com/Yoonsen/Modules/master/module_update.py" > "module_update.py"

In [ ]:
from module_update import update, css, code_toggle, printmd
update('nbtext', overwrite=True)
update('graph_networkx_louvain', overwrite=True)

## Her er noen tilpassede funksjoner

In [ ]:
import requests
def normalize_corpus_dataframe(df):
    colsums = df.sum()
    for x in colsums.index:
        #print(x)
        df[x] = df[x].fillna(0)/colsums[x]
    return True

def urn_coll(word, urns=[], after=5, before=5, limit=1000):
    """Find collocations for a word in a set of books represented by URNs.
    Compute a collocation across the books, or pr. book. Both result in a one column dataframe representing
    the collocations.
    """
    
    if isinstance(urns[0], list):  # urns assumed to be list of list with urn-serial as first element
        urns = [u[0] for u in urns]
    
    r = requests.post("https://api.nb.no/ngram/urncoll", json={'word':word, 'urns':urns, 
                                                                   'after':after, 'before':before, 'limit':limit})
    res = pd.DataFrame.from_dict(r.json(), orient='index')
    if not res.empty:
        res = res.sort_values(by=res.columns[0], ascending = False)
    return res

def urn_coll_pr(word, urns=[], after=5, before=5, limit=1000):
        coll = dict()
        for urn in urns:
            cll = urn_coll(word, [urn], after=after, before=before, limit=limit)
            if not cll.empty:
                coll[urn] = cll
        colls = pd.DataFrame.from_dict(coll, orient='index' ).fillna(0)
        normalize_corpus_dataframe(colls)
        scoll = colls.mean(axis=1)
        res = pd.DataFrame(scoll.sort_values(ascending=False))


Aktiver eller fjern `css`

In [ ]:
css()

Importer moduler

In [ ]:
import nbtext as nb
import graph_networkx_louvain as gnl
import pandas as pd
%matplotlib inline

Slår av advarsler fra IPython

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Notebook for erosmining


Pass på referansene til URN-definisjonene

In [ ]:
urns = nb.get_urns_from_docx('../Materiale erotisk litteratur 1945-1970 liste 27.09.18.docx')
print('Antall URNer: ', len(urns), urns[:5])

Det kvinnelige materialet

In [ ]:
fem_urns = nb.get_urns_from_docx('../Kvinner Materiale erotisk litteratur 1945-1970 liste  08.02.19.docx')
print("Antall: ", len(fem_urns))

In [ ]:
# rekonstruerer metadata for URN-ene
meta = [nb.metadata(u) for u in urns]

In [ ]:
# henter 50 000 ord fra totalen
tot = pd.Series(nb.totals(50000))
tot = tot/tot.sum()

### En hurtigversjon av konkordansekommando

Skriv `konk('bryst')` for å undersøke korpuset. For å se blant URN-er fra kvinner, skriv `konk('bryst', urns=fem_urns)`.

I neste celle defineres kommandoen, med eksempler etter der igjen.

In [ ]:
konk = lambda word, urns = urns, before = 8, after = 4, size = 2: nb.get_urnkonk(word, {'urns':nb.refine_urn(urns,{'words':[word]} )[:30], 'before':before, 'after':after, 'size':size})

## Konkordanser

Bare kopier cellen og kjør flere ganger. Det tar noen sekunder, i og med at korpuset er såpass stort. Effektivisering er under planlegging.

In [ ]:
konk('blusen', urns=fem_urns)

## Næranalyse av en enkelt tekst

En eller flere ordlister for forskjellige formål. 

In [ ]:
# Eksempel fra noen av ordene nevnt i den her sammenhengen
words = """brystene brystet låret lår lemmet presset presse presser stønnet stønne stønner leppene rev rive river flerret flerre flerrer
snerret snerre snerrer kjønnet grotten stivt slapt slapp stive stiv
 kikket kikke kikker stirret stirre stirrer""".split()

In [ ]:
omsorg = """hverandre holdt ømt varsomt følelse følte nærhet nærheten""".split()

In [ ]:
gladord = nb.relaterte_ord('glad, munter')

# Grafanalyse av ordene i en enkelt tekst

Her får du en graf som viser samforekomsten av ord. Det lages en kobling om ordene står i samme avsnitt. Det går også fint å lime en URN fra word-dokumentet.

In [ ]:
print(nb.metadata(urns[30]))

In [ ]:
G = nb.make_network(urns[30], wordbag=words+omsorg)

In [ ]:
gnl.show_graph(G);

In [ ]:
def collcomp(df_column, comp, exp = 1):
    """df_column is a column in a dataframe - a series, while comp is also a series object"""
    res = (df_column**exp/comp).sort_values(ascending=False)
    return res

In [ ]:
import networkx as nx
def make_urn_collocation_graph(target_word, urns, comparo, top = 15, before = 4, after = 4, limit = 1000, exp=1):
        """Make a cascaded network of collocations"""

        firstcoll = urn_coll(target_word, urns, before=before, after=after, limit=limit)
        normalize_corpus_dataframe(firstcoll)
        coll = collcomp(firstcoll[0], comparo, exp = exp)
        target_graf = dict()
        edges = []
        for word in coll[:top].index:
            edges.append((target_word, word))
            if word.isalpha():
                twocoll = urn_coll(word, urns, before=before, after=after,  limit=limit)
                normalize_corpus_dataframe(twocoll)
                coll2 = collcomp(twocoll[0], comparo, exp = exp)
                for w in collcomp(coll2, comparo, exp = exp)[:top].index:
                    if w.isalpha():
                        edges.append((word, w)) 

        target_graph = nx.Graph()
        target_graph.add_edges_from(edges)
        return target_graph

### Kommando for å lage ordsky av en kollokasjon

In [ ]:
collcloud = lambda x: nb.cloud(nb.frame(x/x.sum()))

### Kommandoer for ordskyer og grafer

Se eksemplene under.

In [ ]:
# noen dictionary-objekter som holder dataene, du slipper å lage nye variable for hver gang
# totalen må være laget, men den er med i initialiseringen av notebooken,sammen med URNene. Se øverst.

Gf = dict()
Colls = dict()

def kollmaker_graph(word, urns, top = 20, before = 5, after = 5, graphdict = Gf, comparo = tot, exp = 1):
    graphdict[word] = make_urn_collocation_graph(word, urns, comparo, top = top, before=before, after=after, exp=1) 
    return True

def kollmaker(word, urns, before = 5, after = 5, colldict = Colls, comparo = tot, exp = 1):
    coll = urn_coll(word, urns, before=before, after=after)
    normalize_corpus_dataframe(coll)
    colldict[word] = collcomp(coll[0], comparo, exp=exp)
    return True

In [ ]:
from collections import Counter
graf2cloud = lambda G, n = 30: collcloud(pd.Series(dict(Counter(nx.eigenvector_centrality(G)).most_common(n))))

# Vekstdiagram

For vekstdiagram brukes kommandoen nb.vekstdiagram().plot(). Input til vekstdiagram er en URN sammen med en liste av ord. 

In [ ]:
kollmaker('trusen', urns)

In [ ]:
kollmaker('lemmet', urns)

In [ ]:
kollmaker_graph('lemmet', urns, top = 30)

In [ ]:
kollmaker_graph('trusen', urns, top=25)

In [ ]:
collcloud(Colls['trusen'][:30])

In [ ]:
collcloud(Colls['lemmet'][:30])

In [ ]:
graf2cloud(Gf['trusen'])

In [ ]:
graf2cloud(Gf['lemmet'])

# Analyser tekster med vekstdiagram

# Metadata

Her bygger vi bare opp en struktur for å finne frem i metadata, bare hopp over, men det må stå et sted.

In [ ]:
bokinfo = nb.frame([x[0] for x in meta if x != []])
bokinfo.columns = ['urn', 'forfatter', 'år', 'tittel', 'forlag', 'språk', 'trans']
bokinfo.head()

In [ ]:
finn_bok = lambda y, x = 'forfatter': bokinfo[bokinfo[x].str.contains(y)]

def collwords(x, n=20):
    if not x in Colls:
        kollmaker(x, urns)
    res = list(Colls[x][:n].index)
    return res
        


## Hvordan finne en bok?

Bruk kommandoen `finn_bok('Mykle', 'forfatter')` for å finne alt av Mykle. For å finne en tittel er det bare å legge til 'tittel' som andre parameter `finn_bok('sangen', 'tittel')`. Se eksempler. Parametret 'forfatter' er default, så det kan utelates.

Fra listene er det bare å dobbelklikke en URN for å bruke i vekstdiagram. Se nedenfor.

In [ ]:
finn_bok('Bjørg')

In [ ]:
finn_bok('begjær','tittel')

## Ordsett for vekstdiagrammer

Her kan i prinsippet hva som helst brukes. Data fra kollokasjoner, og andre ting. For å hente ut ordsett fra kollokasjoner bruk `collwords()`, som i `collwords('trusen', 30)` for å hente ut de 30 topp ordene fra kollokasjonen for 'trusen'. Om antallet utelates er default-verdien 20. Hvis det er laget en kollokasjon for 'trusen' brukes den, ellers lages en ny, som så vil bli tilgjengelig under `Colls`.



In [ ]:
collwords('trusen', 30)

Ellers går det også an å lage sin egen liste med ord. Bare skriv en liste med mellomrom innenfor """ """ og avslutt med .split(), som her:

In [ ]:
egne_ord = """munter fornøyd lettet smilte lo ler moro morsomt humor""".split()

Som en tredje variant kan relaterte ord hentes fra galaksene med `nb.relaterte_ord()`

In [ ]:
gladord = nb.relaterte_ord('glad, munter, fornøyd', number=30)
print(gladord)

In [ ]:
voldsord = nb.relaterte_ord('vold, slag, spark, knuse', number = 30)
print(voldsord)

Alle listene kan manipuleres på et eller annet vis til en liste som kan fungere, men greit å bruke dem sånn de er også. Det er mulig å finne ut hvilke ord som gir resultatene nedenfor også.

In [ ]:
voyeur = 'stirret, smøg, myste, kikket, spionerte'.split()
print(voyeur)

# Her noen eksempler

In [ ]:
nb.plot_sammen_vekst(2007050401037,[collwords('ømt'), collwords('naken')]).plot(figsize = (14,6), lw = 8, alpha=0.6)

Bare skriv `print(collwords('ømt'))` og lignende for å se listen over ord i kollokasjonene

In [ ]:
print(collwords('ømt'))

In [ ]:
print(collwords('naken'))

In [ ]:
nb.plot_sammen_vekst(2007072501029,[gladord, voldsord]).plot(figsize = (14,6), lw = 8, alpha=0.6)

In [ ]:
# det er forskjell på de egenkomponerte glad-ordene og de fra relaterte_ord()

nb.plot_sammen_vekst(2007072501029,[egne_ord, gladord]).plot(figsize = (14,6), lw = 8, alpha=0.7)

In [ ]:
finn_bok('Masterson')

In [ ]:
# det er forskjell på de egenkomponerte glad-ordene og de fra relaterte_ord()

nb.plot_sammen_vekst(2013053008089,[gladord, voldsord]).plot(figsize = (14,6), lw = 8, alpha=0.7)

In [ ]:

nb.plot_sammen_vekst(2013053008089,[gladord, voldsord,collwords('slo')], window=8000).plot(figsize = (14,6), lw = 8, alpha=0.7)

In [ ]:
print(collwords('trusa'))

In [ ]:
print(collwords('slo'))